In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler, StringIndexer
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.functions import vector_to_array
from confluent_kafka import Producer
import json

In [3]:
spark = SparkSession.builder \
    .appName("cource_work_fraud_detection") \
    .master("spark://localhost:7077") \
    .config('deploy-mode', 'cluster') \
    .config('spark.executor.memory', '20g') \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 16:23:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/30 16:23:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
spark.stop()

%6|1735462476.073|FAIL|rdkafka#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 4 identical error(s) suppressed)


In [4]:
from spark_to_influxdb import SparkToInfluxDB
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# Read from streaming files
file_path = "file:///opt/spark/data/creditcard.csv"
credit_card_df = spark.read.csv(file_path, header=True, inferSchema=True)

streaming_df = spark.readStream \
    .format("csv") \
    .schema(credit_card_df.schema) \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///opt/spark/data/stream_data/")

# Define InfluxDB connection details
url = "http://localhost:8086"
bucket = "fraud_detection"  # Replace with your InfluxDB bucket
bucket="fraud_detection"
token = "my-token "  # Replace with your InfluxDB token
org = "my_org"  # Replace with your InfluxDB organization

client = influxdb_client.InfluxDBClient(url=url, token=token)
write_api = client.write_api(write_options=SYNCHRONOUS)

def processBatch(df, epoch_id):
    # Get the current time
    current_time = time.time()
    # Write data to InfluxDB
    for row in df.collect():
        point = Point("fraud_detection") \
            .tag("transactin_time", current_time) \
            .field("Time", row.Time) \
            .field("Amount", row.Amount) \
            .field("Class", row.Class)
        write_api.write(bucket, org, point)

    # Write data to Kafka
    p = Producer({'bootstrap.servers': 'localhost:9092'})
    for row in df.collect():
        p.produce('fraud_detection', json.dumps(row.asDict()))
    p.flush()

# just simple write into strem and then in kafka
streaming_df.writeStream \
    .format("memory") \
    .queryName("stream_data") \
    .outputMode("append") \
    .foreachBatch(processBatch) \
    .start() \
    .awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/andrew/Documents/repos/big_data/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/andrew/Documents/repos/big_data/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Do some transformations in the streaming data
# and write it into kafka